This notebook generates the following panels:
- ( ) Summary - Scatter: Open loop vs close loop preferred depth
- ( ) Summary - Scatter: Open loop vs close loop preferred RS
- ( ) Summary - Scatter: Open loop vs close loop preferred of
- ( ) Summary - Scatter: Open loop preferred RS - close loop preferred deph
- ( ) Summary - Scatter: Open loop preferred OF - close loop preferred deph

use env 2p_analysis_cottage

In [ ]:
VERSION = 1

In [ ]:
import os
import sys
import defopt
import pickle
import numpy as np
import pandas as pd
import scipy.stats
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import matplotlib as mpl

mpl.rcParams["pdf.fonttype"] = 42  # save text as text not outlines
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings
import seaborn as sns

from cottage_analysis.depth_analysis.filepath.generate_filepaths import *
from cottage_analysis.imaging.common import find_frames
from cottage_analysis.imaging.common import imaging_loggers_formatting as format_loggers
from cottage_analysis.stimulus_structure import sphere_structure as vis_stim_structure
from cottage_analysis.depth_analysis.plotting.plotting_utils import *
from cottage_analysis.depth_analysis.plotting.basic_vis_plots import *
from cottage_analysis.depth_analysis.depth_preprocess.process_params import *
from cottage_analysis.depth_analysis.depth_preprocess.process_trace import *

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from scipy.stats import spearmanr

In [ ]:
# Load data
root = "/camp/lab/znamenskiyp/home/shared/presentations/PetrTalk202302/open_closeloop_RS_OF/"
save_root = (
    "/camp/lab/znamenskiyp/home/shared/presentations/Cosyne2023/ver"
    + str(VERSION)
    + "/open_loop_summary/"
)
if not os.path.exists(save_root):
    os.makedirs(save_root)

with open(root + "data/gaussian_fit_summary_closeloop.pickle", "rb") as handle:
    close_df_old2 = pickle.load(handle)

with open(root + "data/gaussian_fit_summary_openloop.pickle", "rb") as handle:
    open_df_old2 = pickle.load(handle)

with open(save_root + "gaussian_blob_summary_closeloop_old2.pickle", "wb") as handle:
    pickle.dump(close_df_old2, handle)

with open(save_root + "gaussian_blob_summary_openloop_old2.pickle", "wb") as handle:
    pickle.dump(open_df_old2, handle)

In [ ]:
save_root = (
    "/camp/lab/znamenskiyp/home/shared/presentations/Cosyne2023/ver"
    + str(VERSION)
    + "/open_loop_summary/"
)
if not os.path.exists(save_root):
    os.makedirs(save_root)

rawdata_root = "/camp/lab/znamenskiyp/data/instruments/raw_data/projects/"
root = "/camp/lab/znamenskiyp/home/shared/projects/"
project = "hey2_3d-vision_foodres_20220101"
V1_sessions = [
    ["S20230203", "S20230209", "S20230216", "S20230220"],
    ["S20230206", "S20230210", "S20230213", "S20230214"],
]
summary_dir = root + "/" + project + "/Analysis/summary/"

mouses = ["PZAH8.2i", "PZAH8.2f"]
depth_list = [0.05, 0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4]

columns = [
    "ROI",
    "preferred_depth_idx",
    "log_amplitude",
    "xo_logged",
    "yo_logged",
    "log_sigma_x2",
    "log_sigma_y2",
    "theta",
    "offset",
    "r_sq",
    "preferred_depth_max",
    "preferred_depth_gaussian",
]

for protocol in ["SpheresPermTubeReward", "SpheresPermTubeRewardPlayback"]:
    all_results = pd.DataFrame(columns=columns)
    for k in range(len(V1_sessions)):
        mouse = mouses[k]
        for session in V1_sessions[k]:

            # ----- STEP1: Generate file path -----
            print("---START STEP 1---", "\n", "Getting data filepaths...", flush=True)
            flexilims_session = flz.get_flexilims_session(project_id=project)
            sess_children = get_session_children(
                project=project,
                mouse=mouse,
                session=session,
                flexilims_session=flexilims_session,
            )
            root = Path(flz.PARAMETERS["data_root"]["processed"])

            # ----- STEP1: Generate file path -----
            session_analysis_folder_original = (
                root / project / "Analysis" / mouse / session / "SpheresPermTubeReward"
            )
            session_protocol_folder = root / project / mouse / session / protocol
            session_analysis_folder = (
                root / project / "Analysis" / mouse / session / protocol
            )
            (_, _, _, suite2p_folder, _,) = generate_file_folders(
                project=project,
                mouse=mouse,
                session=session,
                protocol=protocol,
                all_protocol_recording_entries=None,
                recording_no=0,
                flexilims_session=None,
            )

            filename = (
                session_analysis_folder
                / "plane0/gaussian_blob_fit_new_0.1_actual_running.pickle"
            )

            with open(filename, "rb") as handle:
                results_blob = pickle.load(handle)

            # Load depth neurons
            depth_neurons = np.load(
                session_analysis_folder_original / ("plane0/depth_neurons.npy")
            )
            max_depths = np.load(
                session_analysis_folder_original / ("plane0/max_depths_index.npy")
            )

            # Append info to all_results
            results_blob["preferred_depth_max"] = [
                depth_list[int(max_depth)] * 100
                for max_depth in results_blob["preferred_depth_idx"]
            ]
            results_blob = results_blob[columns]
            all_results = pd.concat([all_results, results_blob], ignore_index=True)
            print(session)

    if protocol == "SpheresPermTubeReward":
        sfx = "closeloop"
    else:
        sfx = "openloop"
    with open(f"{save_root}gaussian_blob_summary_{sfx}_new3.pickle", "wb") as handle:
        pickle.dump(all_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
save_root = (
    "/camp/lab/znamenskiyp/home/shared/presentations/Cosyne2023/ver"
    + str(VERSION)
    + "/open_loop_summary/"
)

with open(save_root + "gaussian_blob_summary_closeloop_old2.pickle", "rb") as handle:
    close_old2 = pickle.load(handle)
with open(save_root + "gaussian_blob_summary_openloop_old2.pickle", "rb") as handle:
    open_old2 = pickle.load(handle)
with open(save_root + "gaussian_blob_summary_closeloop_new3.pickle", "rb") as handle:
    close_new3 = pickle.load(handle)
with open(save_root + "gaussian_blob_summary_openloop_new3.pickle", "rb") as handle:
    open_new3 = pickle.load(handle)

close_df = pd.concat([close_old2, close_new3], ignore_index=True)
open_df = pd.concat([open_old2, open_new3], ignore_index=True)
with open(save_root + "gaussian_blob_summary_closeloop_5.pickle", "wb") as handle:
    pickle.dump(close_df, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_root + "gaussian_blob_summary_openloop_5.pickle", "wb") as handle:
    pickle.dump(open_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
close_new3

In [ ]:
# Load data
root = "/camp/lab/znamenskiyp/home/shared/presentations/PetrTalk202302/open_closeloop_RS_OF/"
save_root = (
    "/camp/lab/znamenskiyp/home/shared/presentations/Cosyne2023/ver"
    + str(VERSION)
    + "/open_loop_summary/"
)
if not os.path.exists(save_root):
    os.makedirs(save_root)

with open(save_root + "gaussian_blob_summary_closeloop_5.pickle", "rb") as handle:
    close_df_original = pickle.load(handle)

with open(save_root + "gaussian_blob_summary_openloop_5.pickle", "rb") as handle:
    open_df_original = pickle.load(handle)


# Filter neurons based on the goodness of fit (R2) of openloop
r2_thr = 0.02
select_idx = (close_df_original["r_sq"] > r2_thr) & (open_df_original["r_sq"] > r2_thr)
close_df = close_df_original[select_idx]
open_df = open_df_original[select_idx]

In [ ]:
def set_aspect_ratio(ax, ratio=1):
    x_left, x_right = ax.get_xlim()
    y_low, y_high = ax.get_ylim()
    ax.set_aspect(abs((x_right - x_left) / (y_low - y_high)) * ratio)


# Plot Close loop vs Open loop, preferred depth, preferred RS, preferred OF
rs_color = "g"
of_color = "b"
s = 10
alpha = 0.2
fontsize_dict = {
    "title": 30,
    "xlabel": 30,
    "ylabel": 30,
    "xticks": 20,
    "yticks": 20,
    "legend": 20,
}

# --- FIG 1: Preferred depth, close vs open ---
depth_close = np.array(close_df["preferred_depth_gaussian"])
depth_open = np.array(open_df["preferred_depth_gaussian"])
# plt.figure(figsize=(5,5))
# scatter_plot_fit_line(X=depth_close.reshape(-1,1), y=depth_open.reshape(-1,1),
#                       x2=depth_close.reshape(-1,1),
#                       xlabel='Preferred depth closed loop (cm)',
#                           ylabel='Preferred depth open loop (cm)', n_boots=50000, s=8, alpha=0.2,c='k',
#                       log=True,
#                      fit_line=False,
#                      model=LinearRegression())
# print('Spearman R between close & open loop preferred depth:', spearmanr(depth_close, depth_open))
# plt.xlabel('Preferred depth closed loop (cm)', fontsize=fontsize_dict['xlabel'])
# plt.ylabel('Preferred depth open loop (cm)', fontsize=fontsize_dict['ylabel'])
# plt.xticks(fontsize=fontsize_dict['xticks'])
# plt.yticks(fontsize=fontsize_dict['yticks'])
# plt.gca().set_aspect('equal','box')
# plt.tight_layout(pad=1)
# plt.savefig(save_root + 'preferred_depth_close_vs_open.pdf')

# --- FIG 2: Preferred RS, close vs open ---
rs_close = np.exp(np.array(close_df["xo_logged"].astype("float64")))
rs_open = np.exp(np.array(open_df["xo_logged"].astype("float64")))
plt.figure(figsize=(28 / 2.54, 30 / 2.54))
ax1 = plt.subplot(2, 2, 1)
scatter_plot_fit_line(
    X=rs_close.reshape(-1, 1),
    y=rs_open.reshape(-1, 1),
    x2=rs_close.reshape(-1, 1),
    xlabel="",
    ylabel="",
    n_boots=50000,
    s=s,
    alpha=alpha,
    c=rs_color,
    log=True,
    fit_line=False,
    model=LinearRegression(),
)
print(
    "Spearman R between close & open loop preferred RS:", spearmanr(rs_close, rs_open)
)
plt.xlabel(
    "Preferred running \nspeed closed loop \n(cm/s)", fontsize=fontsize_dict["xlabel"]
)
plt.ylabel(
    "Preferred running speed \n open loop (cm/s)", fontsize=fontsize_dict["ylabel"]
)
plt.xticks(fontsize=fontsize_dict["xticks"])
plt.yticks(fontsize=fontsize_dict["yticks"])
plt.title(
    f"R = {np.round(spearmanr(rs_close, rs_open)[0],2)}",
    loc="right",
    fontsize=fontsize_dict["title"],
)
ax1.set_aspect("equal", "box")
# plt.savefig(save_root + 'preferred_rs_close_vs_open.pdf')

# --- FIG 3: Preferred OF, close vs open ---
of_close = np.exp(np.array(close_df["yo_logged"].astype("float64")))
of_open = np.exp(np.array(open_df["yo_logged"].astype("float64")))
ax2 = plt.subplot(2, 2, 3)
scatter_plot_fit_line(
    X=of_close.reshape(-1, 1),
    y=of_open.reshape(-1, 1),
    x2=of_close.reshape(-1, 1),
    xlabel="",
    ylabel="",
    n_boots=50000,
    s=s,
    alpha=alpha,
    c=of_color,
    log=True,
    fit_line=False,
    model=LinearRegression(),
)
print(
    "Spearman R between close & open loop preferred OF:", spearmanr(of_close, of_open)
)
plt.xlabel(
    "Preferred optic flow \nspeed closed loop \n(degrees/s)",
    fontsize=fontsize_dict["xlabel"],
)
plt.ylabel(
    "Preferred optic flow speed \n open loop (degrees/s)",
    fontsize=fontsize_dict["ylabel"],
)
plt.xticks(fontsize=fontsize_dict["xticks"])
plt.yticks(fontsize=fontsize_dict["yticks"])
plt.title(
    f"R = {np.round(spearmanr(of_close, of_open)[0],2)}",
    loc="right",
    fontsize=fontsize_dict["title"],
)
ax2.set_aspect("equal", "box")
plt.tight_layout(pad=1)
# plt.savefig(save_root + 'preferred_RS_OF_close_vs_open.pdf')

# --- FIG 4: Preferred RS open & OF open - closed preferred depth ---
y = np.log(np.array(close_df["preferred_depth_gaussian"]))
X = np.array(open_df[["xo_logged", "yo_logged"]]).astype(
    "float64"
)  # log preferred RS/OF

rs_closeloop = np.array(np.exp(X[:, 0]))
of_closeloop = np.array(np.exp(X[:, 1]))
depth_closeloop = np.exp(y)
fontsize_dict = {
    "title": 30,
    "xlabel": 30,
    "ylabel": 30,
    "xticks": 20,
    "yticks": 20,
    "legend": 20,
}
s = 13
alpha = 0.2
# plt.figure(figsize=(50/2.54,16/2.54))
ax1 = plt.subplot(2, 2, 2)
rs_2_closeloop = np.logspace(
    np.log(np.min(rs_closeloop)), np.log(np.max(rs_closeloop)), num=1000, base=np.e
).reshape(-1, 1)
scatter_plot_fit_line(
    X=rs_closeloop.reshape(-1, 1),
    y=depth_closeloop.reshape(-1, 1),
    x2=rs_2_closeloop,
    xlabel="Preferred running speed (cm/s)",
    ylabel="Preferred depth (cm)",
    n_boots=50000,
    s=s,
    alpha=alpha,
    c=rs_color,
    log=True,
    fit_line=False,
    model=Ridge(alpha=1),
)
plt.xlabel(
    "Preferred running \n speed open loop \n(cm/s)", fontsize=fontsize_dict["xlabel"]
)
plt.ylabel("Preferred depth \nclosed loop(cm)", fontsize=fontsize_dict["ylabel"])
# plt.xlim([np.min(rs_closeloop)-0.01,np.max(rs_closeloop)+100])
# plt.ylim([np.min(depth_closeloop)-0.1,np.max(depth_closeloop)+100])
plt.xticks(fontsize=fontsize_dict["xticks"])
plt.yticks(fontsize=fontsize_dict["yticks"])
ax1.set_aspect("equal")
# set_aspect_ratio(ax1, ratio=1)
plt.title(
    f"R = {np.round(spearmanr(X[:,0],y)[0],2)}",
    loc="right",
    fontsize=fontsize_dict["title"],
)
# plt.tight_layout(pad=1)
print("Spearman R between RS & preferred depth:", spearmanr(X[:, 0], y))

ax2 = plt.subplot(2, 2, 4, aspect=1)
of_2_closeloop = np.logspace(
    np.log(np.min(of_closeloop)), np.log(np.max(of_closeloop)), num=1000, base=np.e
).reshape(-1, 1)

scatter_plot_fit_line(
    X=of_closeloop.reshape(-1, 1),
    y=depth_closeloop.reshape(-1, 1),
    x2=of_2_closeloop,
    xlabel="Preferred running speed (cm/s)",
    ylabel="Preferred depth (cm)",
    n_boots=50000,
    s=s,
    alpha=alpha,
    c=of_color,
    log=True,
    fit_line=False,
    model=Ridge(alpha=1),
)
plt.xlabel(
    "Preferred optic flow \nspeed open loop\n(degrees/s)",
    fontsize=fontsize_dict["xlabel"],
)
plt.ylabel("Preferred depth \nclosed loop(cm)", fontsize=fontsize_dict["ylabel"])
# plt.xlim([np.min(of_closeloop)-0.002,np.max(of_closeloop)+150])
# plt.ylim([np.min(depth_closeloop)-0.1,np.max(depth_closeloop)+100])
plt.xticks(fontsize=fontsize_dict["xticks"])
plt.yticks(fontsize=fontsize_dict["yticks"])
set_aspect_ratio(ax2, ratio=0.9)
plt.title(
    f"R = {np.round(spearmanr(X[:,1],y)[0],2)}",
    loc="right",
    fontsize=fontsize_dict["title"],
)
plt.tight_layout(pad=0.5)
plt.savefig(save_root + "rs_of_summary.pdf")
print("Spearman R between OF & preferred depth:", spearmanr(X[:, 1], y))

In [ ]:
# --- FIG 4: Preferred RS open & OF open - closed preferred depth ---
y = np.log(np.array(close_df["preferred_depth_gaussian"]))
X = np.array(open_df[["xo_logged", "yo_logged"]]).astype(
    "float64"
)  # log preferred RS/OF

rs_closeloop = np.array(np.exp(X[:, 0]))
of_closeloop = np.array(np.exp(X[:, 1]))
depth_closeloop = np.exp(y)
fontsize_dict = {
    "title": 30,
    "xlabel": 30,
    "ylabel": 30,
    "xticks": 20,
    "yticks": 20,
    "legend": 20,
}
s = 13
alpha = 0.2
plt.figure(figsize=(50 / 2.54, 16 / 2.54))
ax1 = plt.subplot(1, 2, 1)
rs_2_closeloop = np.logspace(
    np.log(np.min(rs_closeloop)), np.log(np.max(rs_closeloop)), num=1000, base=np.e
).reshape(-1, 1)
scatter_plot_fit_line(
    X=rs_closeloop.reshape(-1, 1),
    y=depth_closeloop.reshape(-1, 1),
    x2=rs_2_closeloop,
    xlabel="Preferred running speed (cm/s)",
    ylabel="Preferred depth (cm)",
    n_boots=50000,
    s=s,
    alpha=alpha,
    c=rs_color,
    log=True,
    fit_line=False,
    model=Ridge(alpha=1),
)
plt.xlabel("Preferred running speed\nopen loop(cm/s)", fontsize=fontsize_dict["xlabel"])
plt.ylabel("Preferred depth \nclosed loop(cm)", fontsize=fontsize_dict["ylabel"])
# plt.xlim([np.min(rs_closeloop)-0.01,np.max(rs_closeloop)+100])
# plt.ylim([np.min(depth_closeloop)-0.1,np.max(depth_closeloop)+100])
plt.xticks(fontsize=fontsize_dict["xticks"])
plt.yticks(fontsize=fontsize_dict["yticks"])
ax1.set_aspect("equal")
# set_aspect_ratio(ax1, ratio=1)
plt.title(
    f"R = {np.round(spearmanr(X[:,0],y)[0],2)}",
    loc="right",
    fontsize=fontsize_dict["title"],
)
# plt.tight_layout(pad=1)
print("Spearman R between RS & preferred depth:", spearmanr(X[:, 0], y))

ax2 = plt.subplot(1, 2, 2, aspect=1)
of_2_closeloop = np.logspace(
    np.log(np.min(of_closeloop)), np.log(np.max(of_closeloop)), num=1000, base=np.e
).reshape(-1, 1)

scatter_plot_fit_line(
    X=of_closeloop.reshape(-1, 1),
    y=depth_closeloop.reshape(-1, 1),
    x2=of_2_closeloop,
    xlabel="Preferred running speed (cm/s)",
    ylabel="Preferred depth (cm)",
    n_boots=50000,
    s=s,
    alpha=alpha,
    c=of_color,
    log=True,
    fit_line=False,
    model=Ridge(alpha=1),
)
plt.xlabel(
    "Preferred optic flow speed \nopen loop(degrees/s)",
    fontsize=fontsize_dict["xlabel"],
)
plt.ylabel("Preferred depth \nclosed loop(cm)", fontsize=fontsize_dict["ylabel"])
# plt.xlim([np.min(of_closeloop)-0.002,np.max(of_closeloop)+150])
# plt.ylim([np.min(depth_closeloop)-0.1,np.max(depth_closeloop)+100])
plt.xticks(fontsize=fontsize_dict["xticks"])
plt.yticks(fontsize=fontsize_dict["yticks"])
set_aspect_ratio(ax2, ratio=0.9)
plt.title(
    f"R = {np.round(spearmanr(X[:,1],y)[0],2)}",
    loc="right",
    fontsize=fontsize_dict["title"],
)
plt.tight_layout(pad=2)
plt.savefig(save_root + "rs_of_depth_scatter.pdf")
print("Spearman R between OF & preferred depth:", spearmanr(X[:, 1], y))

In [ ]:
plt.hist(rs_closeloop.reshape(-1, 1))

In [ ]:
sorted(close_df.preferred_depth_max.unique())

In [ ]:
rs_observation = np.load(Path(save_root).parent / "all_running_speeds.npy")
rs_observation = rs_observation[~np.isnan(rs_observation)]
h, b = np.histogram(np.log10(rs_observation), bins=np.arange(0, 3, 0.02))
ax0 = plt.subplot(6, 1, 1)
ax0.bar(b[:-1], h, width=0.02)
ax0.set_xlabel("RS")
xt = [1, 3, 10, 30, 100]
ax0.set_xticks(np.log10(xt))
ax0.set_xticklabels(xt)

of_bin_log_min = -1.5
of_bin_log_max = 3.5
of_bin_num = 11

for id, depth in enumerate([6.0, 19.0, 60.0, 190.0, 600.0][::-1]):
    calc_of = np.rad2deg(rs_observation / depth)
    ax = plt.subplot(6, 1, 2 + id)
    h, b = np.histogram(np.log10(calc_of), bins=np.arange(-1.5, 3.5, 0.02))
    ax.bar(b[:-1], h, width=0.02)

    ax.set_ylabel(depth)

In [ ]:
calc_of = np.rad2deg(rs_closeloop.reshape(-1, 1) / depth_closeloop.reshape(-1, 1))


fig = plt.figure(figsize=(7, 5))
ax = fig.add_subplot(1, 1, 1)
depth_list5 = [6.0, 19.0, 60.0, 190.0, 600.0, 1900.0]
for id, depth in enumerate(depth_list5[:-1]):
    ax.axhline(depth, color="gray", zorder=-10, alpha=0.5)
    calc_of = np.rad2deg(rs_observation[rs_observation > 1] / depth)
    h, b = np.histogram(calc_of, bins=np.logspace(of_bin_log_min, of_bin_log_max, 100))
    ax.bar(
        b[:-1],
        h / h.max() * np.diff(depth_list5)[id] / 2,
        width=np.diff(b),
        bottom=depth,
        color="lightgreen",
        edgecolor="k",
        alpha=0.7,
    )

ax.set_ylabel("Depth (cm)")
ax.set_xlabel(r"Optic flow ($^{\circ}.s^{-1}$)")
ax.scatter(of_closeloop, depth_closeloop, s=13, color="dodgerblue", alpha=0.3)
ax.set_aspect("equal")
plt.loglog()
target = Path(save_root) / "prefered_of_with_shiftedRSdistributions.pdf"
fig.savefig(target, dpi=300)

In [ ]:
str(target)

In [ ]:
calc_of = np.rad2deg(rs_closeloop.reshape(-1, 1) / depth_closeloop.reshape(-1, 1))


fig = plt.figure(figsize=(7, 5))
ax = fig.add_subplot(1, 1, 1)
depth_list5 = np.array([2, 6.0, 19.0, 60.0, 190.0, 600.0, 1900.0])
for sp in [1, 10, 100, 1000]:
    calc_of = np.rad2deg(sp / depth_list5)
    ax.plot(calc_of, depth_list5, color="gray", zorder=-10, alpha=0.5)
for sp in np.nanquantile(rs_observation[rs_observation > 1], [0.05, 0.95]):
    calc_of = np.rad2deg(sp / depth_list5)
    ax.plot(calc_of, depth_list5, color="purple", zorder=-10, alpha=0.5)

ax.set_ylabel("Depth (m)")
ax.set_xlabel(r"Optic flow ($^{\circ}.s^{-1}$)")
ax.scatter(of_closeloop, depth_closeloop, s=13, color="dodgerblue", alpha=0.3)
ax.set_aspect("equal")
plt.loglog()

In [ ]:
plt.plot(out[0])

In [ ]:
d

In [ ]:
depth_y = np.logspace(depth_list5.min() ** (-10), depth_list5.max() ** (-10), 100)
of_bins = np.logspace(of_bin_log_min, of_bin_log_max, 100)
out = np.zeros((len(depth_y), len(of_bins) - 1))
valid = rs_observation[rs_observation > 1]
for id, d in enumerate(depth_y):
    calc_of = np.rad2deg(valid / d)
    out[id], _ = np.histogram(calc_of, bins=of_bins)

In [ ]:
# --- FIG 5: Preferred RS open & OF open - closed preferred depth together---
fontsize_dict = {
    "title": 30,
    "xlabel": 30,
    "ylabel": 30,
    "xticks": 20,
    "yticks": 20,
    "legend": 15,
}
s = 80

sns.color_palette("tab10")

# Filter neurons based on the goodness of fit (R2) of openloop
r2_thr = 0.02
select_idx = (close_df_original["r_sq"] > r2_thr) & (open_df_original["r_sq"] > r2_thr)
close_df = close_df_original[select_idx]
open_df = open_df_original[select_idx]

df = pd.DataFrame(
    columns=[
        "preferred_running_speed_open_loop",
        "preferred_of_speed_open_loop",
        "preferred_depth_gaussian",
    ]
)
df["preferred_depth_gaussian"] = close_df["preferred_depth_gaussian"].astype(float)
df["preferred_running_speed_open_loop"] = np.exp(
    open_df["xo_logged"].to_numpy().astype(float)
)
df["preferred_of_speed_open_loop"] = np.exp(
    open_df["yo_logged"].to_numpy().astype(float)
)
plt.figure(figsize=(25 / 2.45, 30 / 2.45))
ax = sns.scatterplot(
    df,
    x="preferred_running_speed_open_loop",
    y="preferred_of_speed_open_loop",
    hue=np.log(df["preferred_depth_gaussian"]),
    # hue_norm = (np.log(6), np.log(600)),
    palette="cool_r",
    s=s,
)
ax.set_position([0, 0, 1, 1])
sns.despine()
plt.gca().set_aspect("equal", "box")
plt.xscale("log")
plt.yscale("log")
plt.gca().get_legend().remove()
plt.xlabel(
    "Preferred running \n speed open loop (cm/s)", fontsize=fontsize_dict["xlabel"]
)
plt.ylabel(
    "Preferred optic flow speed \n open loop (degrees/s)",
    fontsize=fontsize_dict["ylabel"],
)
norm = mpl.colors.LogNorm(
    np.nanmin(df["preferred_depth_gaussian"]), np.nanmax(df["preferred_depth_gaussian"])
)
norm = mpl.colors.LogNorm(
    np.nanmin(df["preferred_depth_gaussian"]), np.nanmax(df["preferred_depth_gaussian"])
)
sm = plt.cm.ScalarMappable(cmap="cool_r", norm=norm)
sm.set_array([])

# Remove the legend and add a colorbar
cbar = ax.figure.colorbar(sm)
cbar.ax.set_ylabel(
    "Preferred depth (cm)", rotation=270, fontsize=fontsize_dict["legend"]
)
cbar.ax.tick_params(labelsize=fontsize_dict["legend"])
cbar.ax.get_yaxis().labelpad = 25
yticks = cbar.ax.get_yticks()
# cbar.ax.set_yticklabels(np.arange(len(yticks)), np.geomspace(np.nanmin(df["preferred_depth_gaussian"]), np.nanmax(df["preferred_depth_gaussian"]), len(yticks)))

plt.xticks(fontsize=fontsize_dict["xticks"])
plt.yticks(fontsize=fontsize_dict["yticks"])
plt.tight_layout(pad=0.5)
plt.savefig(save_root + "rs_of_depth_scatter_together.pdf")

In [ ]:
# --- FIG 5: Preferred RS close & OF close - closed preferred depth together---
fontsize_dict = {
    "title": 30,
    "xlabel": 30,
    "ylabel": 30,
    "xticks": 20,
    "yticks": 20,
    "legend": 15,
}
s = 80

sns.color_palette("tab10")

# Filter neurons based on the goodness of fit (R2) of openloop
r2_thr = 0.02
select_idx = (close_df_original["r_sq"] > r2_thr) & (open_df_original["r_sq"] > r2_thr)
close_df = close_df_original[select_idx]
open_df = open_df_original[select_idx]

df = pd.DataFrame(
    columns=[
        "preferred_running_speed_open_loop",
        "preferred_of_speed_open_loop",
        "preferred_depth_gaussian",
    ]
)
df["preferred_depth_gaussian"] = close_df["preferred_depth_gaussian"].astype(float)
df["preferred_running_speed_open_loop"] = np.exp(
    close_df["xo_logged"].to_numpy().astype(float)
)
df["preferred_of_speed_open_loop"] = np.exp(
    close_df["yo_logged"].to_numpy().astype(float)
)
plt.figure(figsize=(25 / 2.45, 30 / 2.45))
ax = sns.scatterplot(
    df,
    x="preferred_running_speed_open_loop",
    y="preferred_of_speed_open_loop",
    hue=np.log(df["preferred_depth_gaussian"]),
    # hue_norm = (np.log(6), np.log(600)),
    palette="cool_r",
    s=s,
)
ax.set_position([0, 0, 1, 1])
sns.despine()
plt.gca().set_aspect("equal", "box")
plt.xscale("log")
plt.yscale("log")
plt.gca().get_legend().remove()
plt.xlabel(
    "Preferred running \n speed closed loop (cm/s)", fontsize=fontsize_dict["xlabel"]
)
plt.ylabel(
    "Preferred optic flow speed \n closed loop (degrees/s)",
    fontsize=fontsize_dict["ylabel"],
)
norm = mpl.colors.LogNorm(
    np.nanmin(df["preferred_depth_gaussian"]), np.nanmax(df["preferred_depth_gaussian"])
)
norm = mpl.colors.LogNorm(
    np.nanmin(df["preferred_depth_gaussian"]), np.nanmax(df["preferred_depth_gaussian"])
)
sm = plt.cm.ScalarMappable(cmap="cool_r", norm=norm)
sm.set_array([])

# Remove the legend and add a colorbar
cbar = ax.figure.colorbar(sm)
cbar.ax.set_ylabel(
    "Preferred depth (cm)", rotation=270, fontsize=fontsize_dict["legend"]
)
cbar.ax.tick_params(labelsize=fontsize_dict["legend"])
cbar.ax.get_yaxis().labelpad = 25
yticks = cbar.ax.get_yticks()
# cbar.ax.set_yticklabels(np.arange(len(yticks)), np.geomspace(np.nanmin(df["preferred_depth_gaussian"]), np.nanmax(df["preferred_depth_gaussian"]), len(yticks)))

plt.xticks(fontsize=fontsize_dict["xticks"])
plt.yticks(fontsize=fontsize_dict["yticks"])
plt.tight_layout(pad=0.5)
plt.savefig(save_root + "rs_of_depth_scatter_together_closed_only.pdf")